# Imports

In [ ]:
import os, pickle, json, gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import Counter
import numpy as np, pandas as pd
from sklearn.model_selection import KFold

This Notebook will contain the code necessary for 

In [ ]:
def gen_for_serialisation(path_to_train):
    site_data_files = [x for x in os.listdir(path_to_train)]
    for file in site_data_files:
        #../input/indoor-positioning-traindata/train_data
        f = open(path_to_train +'/'+file, "rb")
        site, train, ground = pickle.load(f)
        f.close()
        yield site,train,ground
def get_x_y_floor(truth_array):
    xs = list()
    ys = list()
    floors = list()
    for lst in truth_array:
        xs.append(lst[0])
        ys.append(lst[1])
        floors.append(lst[2])
    return xs,ys,floors

# Model Definition

Run this code to create model

In [ ]:
def create_model(input_d):
    w1 = int(input_d/2)
    w2 = int(input_d/20)
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape(input_d,1)))
    model.add(layers.Dense(w1, activation='sigmoid'))
    model.add(layers.Dense(w2, activation='sigmoid'))
    model.add(layers.Dense(3, activation='sigmoid'))
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, 
                        beta_2=0.999, epsilon=1e-07, 
                        amsgrad=False,name='Adam'),
                  loss='mse', 
                  metrics=['mean_squared_error'])
    #model.summary()
    return model

# Fitting

In [ ]:
#fit to the neural net
def fit(train_data, target_data, experiment_no, path_to_save):
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=10, shuffle=True)
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(train_data, target_data):
        model = create_model(train_data[0].shape[0])
        history = model.fit(
                            train_data[train], 
                            target_data[train], 
                            batch_size=32,
                            verbose=1, 
                            epochs=100,
                            callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
                            validation_data=(train_data[test],target_data[test]))
        with open('{pts}/{site}_{fold}_NN{exp_no}.pickle'.format(pts=path_to_save,site=site, fold=fold_no, exp_no=experiment_no), 'wb') as f:
            pickle.dump(history.history, f)
        fold_no = fold_no +1
#fits model to x,y and floor coordinates
def fit_model_site(site, train_data, target_data, path_to_save):
    train_data = np.array(train_data)
    train_data = train_data.astype(np.float)
    target_data = np.array(target_data)
    target_data = target_data.astype(np.float)
    #predictors = ['x','y','floor']
    fit(train_data, target_data, "03", path_to_save)

#fits models for x,y, and floor seperately.
def fit_model_site_all_three_model(site, train_data, target_data, path_to_save, exp_no)
    train_data = np.array(train_data)
    xs,ys,floors = get_x_y_floor(target_data)
    target_data = {"xs": xs, "ys": ys, "floors" :floors}
    for target in ["xs", "ys", "floors"]: #maybe list
        fit(train_data, target_data[target], "NN{exp_no}_{name}.pickle".format(exp_no=exp_no, name=target), path_to_save)


# Actual Training

In [ ]:
path_to_save = ""
exp_no = ""
gen = gen_for_serialisation()
for site, train, truth in gen:
    fit_model_site_all_three_model(site, train, truth, path_to_save, exp_no)